In [39]:
# Import Dependencies

from sqlalchemy import create_engine
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [40]:
import_df = pd.read_csv('Resources/Flights_2022_1.csv')


/Applications/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (11,13,78,85,86,93) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
# Making new binary column that tracks delay for both arr and dep

arr_dep_delayed = (import_df['ArrDelayMinutes']>0) | (import_df['DepDelayMinutes']>0)

import_df['Delayed'] = np.where(arr_dep_delayed, 1, 0)

In [42]:
# Creates an integer coded target column for the Machine learning model 

on_time = (import_df['Delayed']==0) & (import_df['Cancelled']==0)
delayed = import_df['Delayed']==1
cancelled = import_df['Cancelled']==1

import_df['Target'] = np.where(on_time, '0', np.where(delayed,'1', np.where(cancelled,'2','')))

In [43]:
[x for x in import_df.columns]

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Originally_Scheduled_Code_Share_Airline',
 'DOT_ID_Originally_Scheduled_Code_Share_Airline',
 'IATA_Code_Originally_Scheduled_Code_Share_Airline',
 'Flight_Num_Originally_Scheduled_Code_Share_Airline',
 'Operating_Airline ',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 '

In [44]:
df= import_df[['Month', 'DayofMonth', 'DayOfWeek', 'Operating_Airline ', 'Origin',
       'Dest', 'Diverted', 'CRSArrTime', 'CRSDepTime', 'ArrTime', 'DepTime',
       'Target', 'OriginCityName', 'DestCityName']].copy()

In [45]:
df=import_df[[
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay',
    'Month', 'DayofMonth', 'DayOfWeek', 'Operating_Airline ', 'Origin',
       'Dest', 'Diverted', 'CRSArrTime', 'CRSDepTime', 'ArrTime', 'DepTime',
       'Target', 'OriginCityName', 'DestCityName','CancellationCode'
    ]]

In [46]:
# Get City/State into seperate columns

df[['Origin_City','Origin_State']] = df['OriginCityName'].str.split(', ',expand=True)
df[['Destination_City','Destination_State']] = df['DestCityName'].str.split(', ',expand=True)

# Remove old column
df.drop(columns=['OriginCityName','DestCityName'],inplace=True)


/Applications/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Applications/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [47]:
df

,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Month,DayofMonth,DayOfWeek,Operating_Airline,Origin,...,CRSArrTime,CRSDepTime,ArrTime,DepTime,Target,CancellationCode,Origin_City,Origin_State,Destination_City,Destination_State
0,NaN,NaN,NaN,NaN,NaN,1,6,4,DL,FLL,...,1419,1126,NaN,NaN,2,A,Fort Lauderdale,FL,New York,NY
1,NaN,NaN,NaN,NaN,NaN,1,6,4,DL,ATL,...,1821,1631,1820.0,1627.0,0,NaN,Atlanta,GA,Fort Lauderdale,FL
2,NaN,NaN,NaN,NaN,NaN,1,6,4,DL,FLL,...,2127,1931,2115.0,1929.0,0,NaN,Fort Lauderdale,FL,Atlanta,GA
3,NaN,NaN,NaN,NaN,NaN,1,6,4,DL,FLL,...,1227,1024,1212.0,1019.0,0,NaN,Fort Lauderdale,FL,Raleigh/Durham,NC
4,NaN,NaN,NaN,NaN,NaN,1,6,4,DL,ATL,...,1142,1117,1131.0,1113.0,0,NaN,Atlanta,GA,Jackson/Vicksburg,MS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563732,NaN,NaN,NaN,NaN,NaN,1,24,1,YV,IAH,...,1414,1240,NaN,NaN,2,D,Houston,TX,Oklahoma City,OK
563733,NaN,NaN,NaN,NaN,NaN,1,24,1,YV,OKC,...,1836,1702,NaN,NaN,2,D,Oklahoma City,OK,Houston,TX
563734,NaN,NaN,NaN,NaN,NaN,1,24,1,YV,ATL,...,1602,1415,NaN,NaN,2,D,Atlanta,GA,Washington,DC
563735,NaN,NaN,NaN,NaN,NaN,1,24,1,YV,IAD,...,1318,1115,NaN,NaN,2,D,Washington,DC,Atlanta,GA


In [48]:
# Fill NA values in Arr and Dep Time columns

df['ArrTime'] = df['ArrTime'].fillna(0)
df['DepTime'] = df['DepTime'].fillna(0)

/Applications/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Applications/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [49]:
df[['CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay']] = df[['CarrierDelay',
                                'WeatherDelay',
                                'NASDelay',
                                'SecurityDelay',
                                'LateAircraftDelay']].fillna(0)

/Applications/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [50]:
# Create list of columns that need encoding

encode_list = [
    'Operating_Airline ',
    'Origin',
    'Dest',
    'Origin_City',
    'Origin_State',
    'Destination_City',
    'Destination_State'
    ]

In [51]:
# create a new df to perform transformations
encoder_df = df.copy()

In [52]:
# Create function to output encoding dictionaries 

def encoder (MasterDF, ColName, Dictionary):
    uni_row_vals =[MasterDF[ColName].unique()][0]
    for x in range(len(uni_row_vals)):
        Dictionary[uni_row_vals[x]] = x

In [53]:
# Integer encode Origin and Destination airports

loc_dict = {}
encoder(encoder_df,'Origin',loc_dict)
encoder_df['Origin']=encoder_df['Origin'].apply(lambda x: loc_dict[x])
encoder_df['Dest']=encoder_df['Dest'].apply(lambda x: loc_dict[x])

In [54]:
# Integer encode Origin and Destination Cities

city_dict = {}
encoder(encoder_df,'Origin_City',city_dict)
encoder_df['Origin_City']=encoder_df['Origin_City'].apply(lambda x: city_dict[x])
encoder_df['Destination_City']=encoder_df['Destination_City'].apply(lambda x: city_dict[x])

In [55]:
# Integer encode Origin and Destination States

state_dict = {}
encoder(encoder_df,'Origin_State',state_dict)
encoder_df['Origin_State']=encoder_df['Origin_State'].apply(lambda x: state_dict[x])
encoder_df['Destination_State']=encoder_df['Destination_State'].apply(lambda x: state_dict[x])

In [56]:
# Integer encode operating airlines
airline_dict = {}
encoder(encoder_df,'Operating_Airline ',airline_dict)
encoder_df['Operating_Airline ']=encoder_df['Operating_Airline '].apply(lambda x: airline_dict[x])

In [61]:
# Function that runs a Random Forest Model and outputs stats on model

def rfmodel_complete (MasterDF,ColumnList,estimators=200,randomstate=52):

    X = MasterDF[ColumnList]
    y = MasterDF['Target']

    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=randomstate)

    # scaler = StandardScaler()

    # X_Scaler = scaler.fit(X_train)


    # X_train_scaled = X_Scaler.transform(X_train)
    # X_test_scaled = X_Scaler.transform(X_test)

    rf_model = RandomForestClassifier(n_estimators=estimators,random_state=randomstate)

    rf_model = rf_model.fit(X_train,y_train)

    rf_predictions = rf_model.predict(X_test)


    cm = confusion_matrix(y_test, rf_predictions)

    # Create a DataFrame from the confusion matrix.
    cm_df = pd.DataFrame(
        cm, index=["On Time", "Delayed","Cancelled"], columns=["On Time (Pred)", "Delayed (Pred)", "Cancelled (Pred)"])

    acc_score = accuracy_score(y_test, rf_predictions)

    # Create a classification report
    class_report = classification_report(y_test, rf_predictions)

    # Calculate feature importance in the Random Forest model.
    importances = rf_model.feature_importances_

    importances_dict = sorted(zip(importances, X.columns), reverse=True)

    return (# Displaying results
        print("\nConfusion Matrix\n"),
        
        print(cm_df),
        
        print(f"\nAccuracy Score : {acc_score}\n"),
        
        print("\nClassification Report\n"),

        print(class_report),
        
        "Feature Importances",
        
        importances_dict
    )

In [31]:
# First run with all features


feature_list_1 = [
    'Month',
    'DayofMonth',
    'DayOfWeek',
    'Operating_Airline ',
    'Origin',
    'Dest',
    'Diverted',
    'CRSArrTime',
    'CRSDepTime',
    'ArrTime',
    'DepTime',
    'Origin_City',
    'Origin_State',
    'Destination_City',
    'Destination_State'
    ]

rfmodel_complete(encoder_df,feature_list_1)


Confusion Matrix

           On Time (Pred)  Delayed (Pred)  Cancelled (Pred)
On Time             69141            5304                 0
Delayed             13487           43988                 6
Cancelled               4              21              8984

Accuracy Score : 0.8664490722673573


Classification Report

              precision    recall  f1-score   support

           0       0.84      0.93      0.88     74445
           1       0.89      0.77      0.82     57481
           2       1.00      1.00      1.00      9009

    accuracy                           0.87    140935
   macro avg       0.91      0.90      0.90    140935
weighted avg       0.87      0.87      0.86    140935



(None,
 None,
 None,
 None,
 None,
 'Feature Importances',
 [(0.2234912678065637, 'ArrTime'),
  (0.22147199286780633, 'DepTime'),
  (0.11390584792576786, 'CRSArrTime'),
  (0.09877279345024242, 'CRSDepTime'),
  (0.08202005392984286, 'DayofMonth'),
  (0.03603828944698542, 'Dest'),
  (0.03525867819244737, 'Destination_City'),
  (0.03359556543030884, 'Origin'),
  (0.03323345107201007, 'Origin_City'),
  (0.030587495475490698, 'Destination_State'),
  (0.030383797803586274, 'DayOfWeek'),
  (0.030318228223006297, 'Operating_Airline '),
  (0.03005004723085815, 'Origin_State'),
  (0.0008724911450837472, 'Diverted'),
  (0.0, 'Month')])

In [15]:
# Second run with removal of CRSArrTime, CRSDepTime, ArrTime, and DepTime as those weighted the model heavily

feature_list_2 = [
    'Month',
    'DayofMonth',
    'DayOfWeek',
    'Operating_Airline ',
    'Origin',
    'Dest',
    'Diverted',
    'Origin_City',
    'Origin_State',
    'Destination_City',
    'Destination_State'
    ]

rfmodel_complete(encoder_df,feature_list_2)


Confusion Matrix

           On Time (Pred)  Delayed (Pred)  Cancelled (Pred)
On Time             52292           20983              1170
Delayed             25700           29515              2266
Cancelled            2304            4104              2601

Accuracy Score : 0.5989143931599673


Classification Report

              precision    recall  f1-score   support

           0       0.65      0.70      0.68     74445
           1       0.54      0.51      0.53     57481
           2       0.43      0.29      0.35      9009

    accuracy                           0.60    140935
   macro avg       0.54      0.50      0.52    140935
weighted avg       0.59      0.60      0.59    140935



(None,
 None,
 None,
 None,
 None,
 'Feature Importances',
 [(0.19804401152793322, 'DayofMonth'),
  (0.12388178712527272, 'Dest'),
  (0.11965773316191546, 'Destination_City'),
  (0.11695521991350682, 'Operating_Airline '),
  (0.1021699572421629, 'Origin'),
  (0.09902705708544723, 'Origin_City'),
  (0.09392361442911198, 'Destination_State'),
  (0.0790339698224714, 'Origin_State'),
  (0.06539737513110658, 'DayOfWeek'),
  (0.0019092745610717575, 'Diverted'),
  (0.0, 'Month')])

In [16]:
# Reintroduce CRS ARR and Dep time (planned arrival and departure time)

feature_list_3 =[
       'DayofMonth',
       'DayOfWeek',
       'Operating_Airline ',
       'Diverted',
       'CRSArrTime',
       'CRSDepTime',
       'Origin_City',
       'Origin_State',
       'Destination_City',
       'Destination_State'
       ]

rfmodel_complete(encoder_df,feature_list_3)


Confusion Matrix

           On Time (Pred)  Delayed (Pred)  Cancelled (Pred)
On Time             57727           16343               375
Delayed             25647           31016               818
Cancelled            2499            3939              2571

Accuracy Score : 0.6479157058218328


Classification Report

              precision    recall  f1-score   support

           0       0.67      0.78      0.72     74445
           1       0.60      0.54      0.57     57481
           2       0.68      0.29      0.40      9009

    accuracy                           0.65    140935
   macro avg       0.65      0.53      0.56    140935
weighted avg       0.65      0.65      0.64    140935



(None,
 None,
 None,
 None,
 None,
 'Feature Importances',
 [(0.17491138207964516, 'CRSArrTime'),
  (0.16700836105264705, 'CRSDepTime'),
  (0.15251587818769682, 'DayofMonth'),
  (0.11115981450789747, 'Destination_City'),
  (0.10562617054130811, 'Origin_City'),
  (0.08350912233193347, 'Destination_State'),
  (0.08060567352101397, 'Origin_State'),
  (0.06928909940402478, 'Operating_Airline '),
  (0.054522184557902, 'DayOfWeek'),
  (0.0008523138159312633, 'Diverted')])

In [17]:
# Model with Actual Arrival and Departure Times

feature_list_4 = [
        'DayofMonth',
        'DayOfWeek',
        'Operating_Airline ',
        'Diverted',
        'ArrTime',
        'DepTime',
        'Origin_City',
        'Origin_State',
        'Destination_City',
        'Destination_State'
        ]

rfmodel_complete(encoder_df,feature_list_4)


Confusion Matrix

           On Time (Pred)  Delayed (Pred)  Cancelled (Pred)
On Time             62352           12093                 0
Delayed             23420           34052                 9
Cancelled               0              28              8981

Accuracy Score : 0.7477560577571221


Classification Report

              precision    recall  f1-score   support

           0       0.73      0.84      0.78     74445
           1       0.74      0.59      0.66     57481
           2       1.00      1.00      1.00      9009

    accuracy                           0.75    140935
   macro avg       0.82      0.81      0.81    140935
weighted avg       0.75      0.75      0.74    140935



(None,
 None,
 None,
 None,
 None,
 'Feature Importances',
 [(0.2659434620405917, 'DepTime'),
  (0.24711678374795726, 'ArrTime'),
  (0.09019876314567792, 'DayofMonth'),
  (0.08980220867479816, 'Destination_City'),
  (0.08331673839756659, 'Origin_City'),
  (0.067367975420559, 'Destination_State'),
  (0.06399005440807322, 'Origin_State'),
  (0.058995341798787224, 'Operating_Airline '),
  (0.03244755361679545, 'DayOfWeek'),
  (0.0008211187491933908, 'Diverted')])

In [38]:
fl5 = [
    'Month',
    'DayofMonth',
    'DayOfWeek',
    'Operating_Airline ',
    'Origin',
    'Dest',
    'Diverted',
    'CRSArrTime',
    'CRSDepTime',
    'ArrTime',
    'DepTime',
    'Origin_City',
    'Origin_State',
    'Destination_City',
    'Destination_State',
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]


df[fl5].dtypes

Month                   int64
DayofMonth              int64
DayOfWeek               int64
Operating_Airline      object
Origin                 object
Dest                   object
Diverted              float64
CRSArrTime              int64
CRSDepTime              int64
ArrTime               float64
DepTime               float64
Origin_City            object
Origin_State           object
Destination_City       object
Destination_State      object
CarrierDelay          float64
WeatherDelay          float64
NASDelay              float64
SecurityDelay         float64
LateAircraftDelay     float64
dtype: object

In [ ]:
fl5 = [
    'Month',
    'DayofMonth',
    'DayOfWeek',
    'Operating_Airline ',
    'Origin',
    'Dest',
    'Diverted',
    'CRSArrTime',
    'CRSDepTime',
    'ArrTime',
    'DepTime',
    'Origin_City',
    'Origin_State',
    'Destination_City',
    'Destination_State',
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]


df[fl5].dtypes

Month                   int64
DayofMonth              int64
DayOfWeek               int64
Operating_Airline      object
Origin                 object
Dest                   object
Diverted              float64
CRSArrTime              int64
CRSDepTime              int64
ArrTime               float64
DepTime               float64
Origin_City            object
Origin_State           object
Destination_City       object
Destination_State      object
CarrierDelay          float64
WeatherDelay          float64
NASDelay              float64
SecurityDelay         float64
LateAircraftDelay     float64
dtype: object

In [60]:
fl5 = [
    'Month',
    'DayofMonth',
    'DayOfWeek',
    'Operating_Airline ',
    'Origin',
    'Dest',
    'Diverted',
    'CRSArrTime',
    'CRSDepTime',
    'ArrTime',
    'DepTime',
    'Origin_City',
    'Origin_State',
    'Destination_City',
    'Destination_State',
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]

rfmodel_complete(encoder_df,fl5)


Confusion Matrix

           On Time (Pred)  Delayed (Pred)  Cancelled (Pred)
On Time             70760            3685                 0
Delayed             17630           39842                 9
Cancelled               1              23              8985

Accuracy Score : 0.8485259162025047


Classification Report

              precision    recall  f1-score   support

           0       0.80      0.95      0.87     74445
           1       0.91      0.69      0.79     57481
           2       1.00      1.00      1.00      9009

    accuracy                           0.85    140935
   macro avg       0.90      0.88      0.89    140935
weighted avg       0.86      0.85      0.84    140935



(None,
 None,
 None,
 None,
 None,
 'Feature Importances',
 [(0.17694192842261805, 'ArrTime'),
  (0.1741295486781566, 'DepTime'),
  (0.11037105991977182, 'CarrierDelay'),
  (0.07033981629030445, 'CRSArrTime'),
  (0.06572726642682851, 'CRSDepTime'),
  (0.06551164428501575, 'NASDelay'),
  (0.05764509896215839, 'LateAircraftDelay'),
  (0.056551714960842776, 'DayofMonth'),
  (0.03105602602667109, 'Dest'),
  (0.0303209246923872, 'Destination_City'),
  (0.027785606926793948, 'Origin'),
  (0.027206996331785586, 'Origin_City'),
  (0.0258108335656627, 'Destination_State'),
  (0.02425325845150263, 'Operating_Airline '),
  (0.024061749956408595, 'Origin_State'),
  (0.023389023233077857, 'DayOfWeek'),
  (0.007997789488050558, 'WeatherDelay'),
  (0.0007236009076534593, 'Diverted'),
  (0.00017611247431002723, 'SecurityDelay'),
  (0.0, 'Month')])

In [62]:
fl6 = [
    'DayofMonth',
    'DayOfWeek',
    'Operating_Airline ',
    'Origin',
    'Dest',
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]

rfmodel_complete(encoder_df,fl6)


Confusion Matrix

           On Time (Pred)  Delayed (Pred)  Cancelled (Pred)
On Time             58546           13804              2095
Delayed             20409           35215              1857
Cancelled            3287            2096              3626

Accuracy Score : 0.6910064923546315


Classification Report

              precision    recall  f1-score   support

           0       0.71      0.79      0.75     74445
           1       0.69      0.61      0.65     57481
           2       0.48      0.40      0.44      9009

    accuracy                           0.69    140935
   macro avg       0.63      0.60      0.61    140935
weighted avg       0.69      0.69      0.69    140935



(None,
 None,
 None,
 None,
 None,
 'Feature Importances',
 [(0.28006377653279035, 'Dest'),
  (0.2637839061917631, 'Origin'),
  (0.14073936771103956, 'CarrierDelay'),
  (0.08368189815610687, 'NASDelay'),
  (0.07495047933884628, 'LateAircraftDelay'),
  (0.06582473788378508, 'DayofMonth'),
  (0.060493716354469616, 'Operating_Airline '),
  (0.018828014889863822, 'DayOfWeek'),
  (0.011124722395746798, 'WeatherDelay'),
  (0.0005093805455884666, 'SecurityDelay')])

In [63]:
fl7 = [
    'CRSArrTime',
    'CRSDepTime',
    'DayofMonth',
    'DayOfWeek',
    'Operating_Airline ',
    'Origin',
    'Dest',
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]

rfmodel_complete(encoder_df,fl7)


Confusion Matrix

           On Time (Pred)  Delayed (Pred)  Cancelled (Pred)
On Time             67021            6673               751
Delayed             23524           33100               857
Cancelled            4150            1644              3215

Accuracy Score : 0.733217440664136


Classification Report

              precision    recall  f1-score   support

           0       0.71      0.90      0.79     74445
           1       0.80      0.58      0.67     57481
           2       0.67      0.36      0.46      9009

    accuracy                           0.73    140935
   macro avg       0.72      0.61      0.64    140935
weighted avg       0.74      0.73      0.72    140935



(None,
 None,
 None,
 None,
 None,
 'Feature Importances',
 [(0.16307116911330602, 'CRSArrTime'),
  (0.15498432232244133, 'CRSDepTime'),
  (0.13075293523700315, 'Origin'),
  (0.13015502929281805, 'Dest'),
  (0.11383790949973349, 'CarrierDelay'),
  (0.0851816218136928, 'DayofMonth'),
  (0.06721102876875835, 'NASDelay'),
  (0.060440174008858406, 'LateAircraftDelay'),
  (0.05762559068337515, 'Operating_Airline '),
  (0.027690107276919833, 'DayOfWeek'),
  (0.008711996303257622, 'WeatherDelay'),
  (0.00033811567983584826, 'SecurityDelay')])